# Decision Tree to classify the priority of an incident

The Incident Management dataset has about 141712 records of 24918 incidents. Each state of the incident is being captured as an individual record with few exceptions where the closed state of an incident is recorded more than once. With the help of the below segment of the code, we load and clean the Incident Management data so that only one record representing the truly closed state per incident is obtained.

------------------------------------------------------------------------------------------------------------------------------

##### Create a spark session and load the Incident Management Data set

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('IMMLDDT').getOrCreate()

In [3]:
df = spark.read.csv('incident_event_log.csv',inferSchema=True,header=True)

------------------------------------------------------------------------------------------------------------------------------

##### Data pre-processing

In [4]:
# The data set has multiple states(New, Active, Awaiting user info, Resolved, Closed etc. ) of an incident. With the help 
# of the below command, we are just filtering one record per incident, that has the truly closed state of the incident. 

df_unique_incidents=df.filter("incident_state=='Closed'").sort("sys_mod_count",ascending=False).dropDuplicates(["number"])

In [5]:
# Selecting the dependent and the independent variables that are identified as most useful attributes to make predictions

data=df_unique_incidents.select(['caller_id','opened_by','location','category','subcategory',
                                 'u_symptom','assignment_group','priority'])

In [6]:
data=data.dropna()

In [7]:
# Create a 70-30 train test split

train_data,test_data=data.randomSplit([0.7,0.3])

------------------------------------------------------------------------------------------------------------------------------

### Building the Decision Tree Classifier

In [8]:
# Import the required libraries

from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline

In [9]:
# Use StringIndexer to convert the categorical columns to hold numerical data

caller_id_indexer = StringIndexer(inputCol='caller_id',outputCol='caller_id_index',handleInvalid='keep')
opened_by_indexer = StringIndexer(inputCol='opened_by',outputCol='opened_by_index',handleInvalid='keep')
location_indexer = StringIndexer(inputCol='location',outputCol='location_index',handleInvalid='keep')
category_indexer = StringIndexer(inputCol='category',outputCol='category_index',handleInvalid='keep')
subcategory_indexer = StringIndexer(inputCol='subcategory',outputCol='subcategory_index',handleInvalid='keep')
u_symptom_indexer = StringIndexer(inputCol='u_symptom',outputCol='u_symptom_index',handleInvalid='keep')
assignment_group_indexer = StringIndexer(inputCol='assignment_group',outputCol='assignment_group_index',handleInvalid='keep')
priority_indexer = StringIndexer(inputCol='priority',outputCol='priority_index',handleInvalid='keep')

In [10]:
# Vector assembler is used to create a vector of input features

assembler = VectorAssembler(inputCols=['caller_id_index','opened_by_index','location_index','category_index',
                                       'subcategory_index','u_symptom_index','assignment_group_index'],
                            outputCol="features")

In [11]:
# Create an object for the Logistic Regression model
# Use the parameter maxBins and assign a value that is equal to or more than the number of categories in any sigle feature

dt_model = DecisionTreeClassifier(labelCol='priority_index',maxBins=5000)

In [12]:
# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data

pipe = Pipeline(stages=[caller_id_indexer,opened_by_indexer,location_indexer,category_indexer,subcategory_indexer,
                        u_symptom_indexer,assignment_group_indexer,priority_indexer,assembler,dt_model])

In [13]:
# It took 8 minutes for this step to execute

fit_model=pipe.fit(train_data)

In [14]:
# Store the results in a dataframe

results = fit_model.transform(test_data)

In [15]:
results.select(['priority_index','prediction']).show()

+--------------+----------+
|priority_index|prediction|
+--------------+----------+
|           0.0|       0.0|
|           0.0|       0.0|
|           0.0|       0.0|
|           0.0|       0.0|
|           0.0|       0.0|
|           0.0|       0.0|
|           0.0|       0.0|
|           0.0|       0.0|
|           0.0|       0.0|
|           0.0|       0.0|
|           0.0|       0.0|
|           0.0|       0.0|
|           3.0|       0.0|
|           0.0|       0.0|
|           0.0|       0.0|
|           0.0|       0.0|
|           0.0|       0.0|
|           0.0|       0.0|
|           0.0|       0.0|
|           0.0|       2.0|
+--------------+----------+
only showing top 20 rows



-------------------------------------------------------------------------------------------------------------------------------

##### Evaluating the model

In [16]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [17]:
ACC_evaluator = MulticlassClassificationEvaluator(
    labelCol="priority_index", predictionCol="prediction", metricName="accuracy")

In [18]:
accuracy = ACC_evaluator.evaluate(results)

In [19]:
print("The accuracy of the decision tree classifier is {}".format(accuracy))

The accuracy of the decision tree classifier is 0.9342070072211821
